In [5]:
#Read in dataset
import json

businesses = {}
reviews = {}
users = {}

cat_ref = ['Food', 'Bagels', 'Bakeries', 'Beer, Wine & Spirits', 'Breweries', 'Bubble Tea', 'Butcher', 'CSA', 'Coffee & Tea', 'Convenience Stores', 'Desserts', 'Do-It-Yourself Food', 'Donuts', 'Farmers Market', 'Food Delivery Services', 'Food Trucks', 'Gelato', 'Grocery', 'Ice Cream & Frozen Yogust', 'Internet Cafes', 'Juice Bars & Smoothies', 'Pretzels', 'Shaved Ice', 'Specialty Food', 'Candy Stores', 'Cheese Shops', 'Chocolatiers & Shops', 'Ethnic Food', 'Fruits & Veggies', 'Health Markets', 'Herbs & Spices', 'Meat Shops', 'Seafood Markets', 'Street Vendors', 'Tea Rooms', 'Wineries', 'Afghan', 'African', 'American (New)', 'American (Traditional)', 'Arabian', 'Argentine', 'Armenian', 'Asian Fusion', 'Australian', 'Austrian', 'Bangladeshi', 'Barbeque', 'Basque', 'Belgian', 'Brasseries', 'Brazilian', 'Breakfast & Brunch', 'British', 'Buffets', 'Burgers', 'Burmese', 'Cafes', 'Cafeteria', 'Cajun/Creole', 'Cambodian', 'Caribbean', 'Catalan', 'Cheesesteaks', 'Chicken Wings', 'Chinese', 'Comfort Food', 'Creperies', 'Cuban', 'Czech', 'Delis', 'Diners', 'Ethiopian', 'Fast Food', 'Filipino', 'Fish & Chips', 'Fondue', 'Food Court', 'Food Stands', 'French', 'Gastropubs', 'German', 'Gluten-Free', 'Greek', 'Halal', 'Hawaiian', 'Himalayan/Nepalese', 'Hot Dogs', 'Hot Pot', 'Hungarian', 'Iberian', 'Indian', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Korean', 'Kosher', 'Laotian', 'Latin American', 'Live/Raw Food', 'Malaysian', 'Meditteranean', 'Mexican', 'Middle Eastern', 'Modern European', 'Mongolian', 'Pakistani', 'Persian/Iranian', 'Peruvian', 'Pizza', 'Polish', 'Portuguese', 'Russian', 'Salad', 'Sandwiches', 'Scandinavian', 'Scottish', 'Seafood', 'Singaporean', 'Slovakian', 'Soul Food', 'Soup', 'Southern', 'Spanish', 'Steakhouses', 'Sushi Bars', 'Taiwanese', 'Tapas Bars', 'Tapas/Small Plates', 'Tex-Mex', 'Thai', 'Turkish', 'Ukranian', 'Vegan', 'Vegetarian', 'Vietnamese', 'Columbian', 'Salvadoran', 'Venezuelan', 'Egyptian', 'Lebanese', 'Dominican', 'Haitian', 'Puerto Rican', 'Trinidadian', 'Cantonese', 'Dim Sum', 'Shanghainese', 'Szechuan', 'Senegalese', 'South African', 'Restaurants'
]

with open('/Users/Maximus/Desktop/W209_DB/yelp_academic_dataset.json') as datafile:
    for i in datafile:
        data = json.loads(i)
        if data['type']=='business':
            good_cats = [x for x in data['categories'] if x in cat_ref]
            if good_cats:
                businesses[data['business_id']]=data
        elif data['type']=='review':
            reviews[data['business_id']+data['user_id']]=data
        elif data['type']=='user':
            users[data['user_id']]=data

In [6]:
#6390 total business 
import math
import numpy as np
import pandas as pd

c_test = []
for i in list(businesses)[0:len(businesses)-1]:
    for j in range(len(businesses[i]['categories'])):
        c_test.append(businesses[i]['categories'][j].encode('utf8'))

feature_list = ['schools', 'city', 'name', 'review_count']
feature_list =feature_list + list(np.unique(c_test))

d_data = pd.DataFrame(0, index=np.arange(6391), columns=feature_list)

c = 0
#for i in list(businesses)[0:len(businesses)]:
for i in list(businesses)[0:len(businesses)]:
    for j in range(len(businesses[i]['schools'])):
        d_data.ix[c,'city'] = businesses[i]['city']   
        d_data.ix[c,'name'] = businesses[i]['name']
        d_data.ix[c, 'review_count'] = businesses[i]['review_count']
        d_data.ix[c, 'schools'] = businesses[i]['schools'][j] 
        for k in range(len(businesses[i]['categories'])):
            l = feature_list.index(businesses[i]['categories'][k])
            d_data.ix[c,l] = businesses[i]['review_count'] 
        c = c + 1
print d_data.iloc[0:10,1:20]       

           city               name  review_count  Active Life  Afghan  \
0     Cambridge    Brookline Lunch           185            0       0   
1     Cambridge    Brookline Lunch           185            0       0   
2          Troy  Notty Pine Tavern             6            0       0   
3  Philadelphia      Pitruco Pizza            42            0       0   
4     Greenbelt            Wendy's             4            0       0   
5       Houston      Ben & Jerry's            12            0       0   
6     Champaign   Papa Del's Pizza           140            0       0   
7        Austin           7 Eleven             9            0       0   
8      New York       Elsabet Cafe            13            0       0   
9     Lafayette        TGI Fridays             7            0       0   

   African  American (New)  American (Traditional)  Antiques  Appliances  \
0        0               0                       0         0           0   
1        0               0                  

In [3]:
print len(d_data.iloc[0,:]) 
print c_test[:10]

215
['Middle Eastern', 'Diners', 'Restaurants', 'Italian', 'Pizza', 'Restaurants', 'Food', 'Pizza', 'Food Trucks', 'Restaurants']


In [7]:
#Necessary step
#Aggregate...find the average rating per category for each school
new_feature_list =['schools'] + list(np.unique(c_test))
meta_data = pd.DataFrame(0, index=np.arange(30), columns=new_feature_list)
sch =  d_data.ix[:,0].unique()
meta_data.ix[0:28, 0] = sch[0:29]

c = 0
for i in sch:
    k = 0
    for j in new_feature_list[1:]:
        t = d_data.ix[(d_data['schools'] == i) & d_data[j]<>0, [j]].mean()[0]
        if math.isnan(t):
            meta_data.ix[c, k+1] = 0
        else: meta_data.ix[c, k+1] = t
        k = k + 1
    c = c + 1

print len(meta_data.iloc[0,:])

212


In [8]:
#Output a category list then manually remove those that didn't fit food category
print meta_data.iloc[:,0] # School names

#Load rankings
for i in range(29):
    #print meta_data.ix[i,0]
    a = np.array(meta_data.ix[i,1:])
    loc = list(a.argsort()[-10:][::-1])
    loc = map(lambda x: x + 1, loc)
    print meta_data.ix[i,loc]

temp = list(np.unique(c_test))
writeout = pd.DataFrame(temp)

#for i in range(len(temp)):
    #writeout.ix[i] = list(np.unique(c_test))[i]

#print type(meta_data.columns)
writeout.to_csv('Category_names_24.txt', sep='\t', index=False)


0          Massachusetts Institute of Technology
1                             Harvard University
2               Rensselaer Polytechnic Institute
3                     University of Pennsylvania
4          University of Maryland - College Park
5                                Rice University
6      University of Illinois - Urbana-Champaign
7                   University of Texas - Austin
8                            Columbia University
9                              Purdue University
10          University of California - San Diego
11                              Brown University
12    University of North Carolina - Chapel Hill
13          University of California at Berkeley
14             University of Southern California
15                          Princeton University
16                    Carnegie Mellon University
17        University of California - Los Angeles
18                      University of Washington
19                           Stanford University
20       California 

In [9]:
#Upload a list of categories to exclude
text_file = open("/Users/Maximus/Desktop/W209_DB/remove.txt", "r")
lines = text_file.readlines()
word = []
for i in lines:
    word.append(i.strip())

In [10]:
campus_key = {
'umich': 'University of Michigan - Ann Arbor',
'penn': 'University of Pennsylvania',
'harvard': 'Harvard University',
'cal': 'University of California at Berkeley',
'umass': 'University of Massachusetts - Amherst',
'vt': 'Virginia Tech',
'unc': 'University of North Carolina - Chapel Hill',
'uw': 'University of Washington',
'utt': 'University of Texas - Austin',
'usc': 'University of Southern California',
'ucla': 'University of California - Los Angeles',
'ucsd': 'University of California - San Diego',
'ill': 'University of Illinois - Urbana-Champaign',
'purdue': 'Purdue University',
'princeton': 'Princeton University',
'mit': 'Massachusetts Institute of Technology',
'harveymudd': 'Harvey Mudd College',
'stanford': 'Stanford University',
'carnegie': 'Carnegie Mellon University',
'columbia': 'Columbia University',
'cornell': 'Cornell University',
'gt': 'Georgia Institute of Technology',
'rice': 'Rice University',
'rensselaer': 'Rensselaer Polytechnic Institute',
'brown': 'Brown University',
'caltech': 'California Institute of Technology',
'cp': 'California Polytechnic State University',
'maryland': 'University of Maryland - College Park',
'waterloo': 'University of Waterloo'}

campus_name = {}
for key, value in campus_key.iteritems():
    campus_name[value] = key

In [9]:
#Write Out text input files for d3
new_col = []

    #new_col.append(meta_data.iloc[i,0].encode('utf8'))  #School names
    #new_col.append(meta_data.iloc[i,0].encode('utf8')+('_count'))

for i in range(29):
    new_col.append( campus_name[meta_data.iloc[i,0]] )
    new_col.append( campus_name[meta_data.iloc[i,0]] + ('_count'))

a_size = 30
final_data  = pd.DataFrame(0, index=np.arange(a_size), columns=new_col)
output_data = pd.DataFrame(0, index=np.arange(10), columns=new_col)
#Final data is the result frame

for i in range(29):
    a = np.array(meta_data.ix[i,1:])
    loc = list(a.argsort()[-a_size:][::-1])
    loc = map(lambda x: x + 1, loc)
    final_data.iloc[:,i*2] = list(meta_data.columns.values[loc])  #Upload column names
    final_data.iloc[:,i*2+1] = list(meta_data.ix[i,loc])
    #print meta_data.ix[i,loc]

#Create another dataframe; move the data input it only if it was in the list
#print final_data
#print final_data['penn']

for i in range(29):
    k = 0
    for count, item in enumerate(final_data.iloc[:,i*2]):
        if (item not in word) and (k < 10):
            output_data.iloc[k, i*2  ] = item 
            output_data.iloc[k, i*2+1] = final_data.iloc[count, i*2+1]
            k = k +1
        elif k == 10:    #As soon as an error, will break out the loop
            break
#print output_data['cal']
writeout = output_data
writeout.to_csv('d3_datav2.txt', sep='\t', index=False)

# Code below is additional data exploration process

In [7]:
#Top food categories for each school
for i in range(29):
    #print meta_data.ix[i,0]
    a = np.array(meta_data.ix[i,1:])
    loc = list(a.argsort()[-10:][::-1])
    loc = map(lambda x: x + 1, loc)
    print meta_data.ix[i,loc]

Cajun/Creole                  969
Tex-Mex                       969
Southern                      675
Desserts                      339
Burgers                   299.667
French                    276.333
Irish                         248
Breweries                 243.667
Juice Bars & Smoothies        243
African                       219
Name: 0, dtype: object
Cajun/Creole                  969
Tex-Mex                       969
Southern                      675
Desserts                      339
Burgers                   299.667
French                    276.333
Irish                         248
Breweries                 243.667
Juice Bars & Smoothies        243
African                       219
Name: 1, dtype: object
Mediterranean       43
Sushi Bars          41
Lounges             32
Hot Dogs            29
Mexican             29
Indian            28.2
Middle Eastern      27
Vegan               27
Desserts            27
Japanese          26.5
Name: 2, dtype: object
Buffets              

In [18]:
print meta_data.iloc[:,0]

0          Massachusetts Institute of Technology
1                             Harvard University
2               Rensselaer Polytechnic Institute
3                     University of Pennsylvania
4          University of Maryland - College Park
5                                Rice University
6      University of Illinois - Urbana-Champaign
7                   University of Texas - Austin
8                            Columbia University
9                              Purdue University
10          University of California - San Diego
11                              Brown University
12    University of North Carolina - Chapel Hill
13          University of California at Berkeley
14             University of Southern California
15                          Princeton University
16                    Carnegie Mellon University
17        University of California - Los Angeles
18                      University of Washington
19                           Stanford University
20       California 

In [41]:
print list(meta_data.columns.values)[0:2]

['schools', 'Active Life']


In [14]:
#Sample on how to output result file

#temp = list(np.unique(c_test))
writeout = final_data
writeout.to_csv('Category_namesv3.txt', sep='\t', index=False)
print 'Final version'

Nano


In [12]:
#Sample on how to output result file
#a = meta_data.ix[0,:]
#writeout = pd.DataFrame(0, index=np.arange(300), columns=['Category'])
temp = list(np.unique(c_test))
writeout = pd.DataFrame(temp)

#for i in range(len(temp)):
    #writeout.ix[i] = list(np.unique(c_test))[i]

#print type(meta_data.columns)
writeout.to_csv('Category_names.txt', sep='\t', index=False)
print 'Nano'

Nano
